In [1]:
# Import the necessary libraries 
# Recall highly correlated column (base and exercised options value) is our highly corr. column
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np
import pandas as pd

# Plotting library
from matplotlib import pyplot

# import function to split data into train and test dfs
from sklearn.model_selection import train_test_split

# Optimization module in scipy
from scipy import optimize

# Import logistic regression model from SK Learn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Import glob as glob, used to consolidate data files
import glob

In [2]:
# os.chdir('C:\\Users\\mczerwinski\\repos\\BudgetPredict')
# # Set working directory to the data folder so you can correctly read in the csv files
# %cd data
# %cd test_data

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\belincoln\\repos\\BudgetPredict'

In [17]:
# read all csv files within the test_data folder. This should be 2008-2018 (11 years), of DHS contract award data
all_files = glob.glob("../data/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, header = 0, usecols = ['contract_transaction_unique_key',
                        'contract_award_unique_key',
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award',
                        'action_date'],
                 dtype = {'contract_transaction_unique_key':'str','contract_transaction_unique_key':'str',
                          'federal_action_obligation': 'float','total_dollars_obligated': 'float', 
                          'base_and_exercised_options_value': 'float', 'current_total_value_of_award': 'float', 
                          'base_and_all_options_value': 'float','potential_total_value_of_award': 'float',
                          'action_date':'str'})
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)


In [4]:
df.head()

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date
0,9700_-NONE-_SPE7L119P4448_0_-NONE-_0,CONT_AWD_SPE7L119P4448_9700_-NONE-_-NONE-,526.78,526.78,526.78,526.78,526.78,526.78,2019-06-19
1,9700_-NONE-_SPE7L119P4463_0_-NONE-_0,CONT_AWD_SPE7L119P4463_9700_-NONE-_-NONE-,3796.95,3796.95,3796.95,3796.95,3796.95,3796.95,2019-06-19
2,9700_-NONE-_SPE7L119P4483_0_-NONE-_0,CONT_AWD_SPE7L119P4483_9700_-NONE-_-NONE-,97.85,97.85,97.85,97.85,97.85,97.85,2019-06-19
3,9700_-NONE-_SPE7L119P4489_0_-NONE-_0,CONT_AWD_SPE7L119P4489_9700_-NONE-_-NONE-,5602.40,5602.40,5602.40,5602.40,5602.40,5602.40,2019-06-19
4,9700_-NONE-_SPE7L119V7674_P00001_-NONE-_0,CONT_AWD_SPE7L119V7674_9700_-NONE-_-NONE-,-20406.72,0.00,-20406.72,0.00,-20406.72,0.00,2019-06-19


In [18]:
# This is what we are using for our rate of Execution
df['Percent Cumulatively Obligated over potential total value of award'] = df['total_dollars_obligated'] / df['potential_total_value_of_award']

# Create Indicator Variable
df['Indicator'] = df['federal_action_obligation']<-1000

df_deob = df.loc[df['Indicator'] == 1]

# df_ind = df.copy().drop(columns = [c for c in df_deob.columns if c not in ['Indicator', 'contract_award_unique_key']])
# #Create indicator for 

# df_ind.head()

df_deob.drop(columns = [c for c in df_deob.columns if c not in ['Indicator', 'contract_award_unique_key']], inplace = True)

df_deob = df_deob.drop_duplicates()

df_ever_deob = df.merge(df_deob, how = 'left', right_on = 'contract_award_unique_key', left_on = 'contract_award_unique_key')

df_ever_deob = df_ever_deob.rename(columns = {'Indicator_x': 'Transaction_Deob', 'Indicator_y': 'Contract_Deob'})
df_ever_deob.head()




,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_Deob
0,9700_-NONE-_SPE7L119P4448_0_-NONE-_0,CONT_AWD_SPE7L119P4448_9700_-NONE-_-NONE-,526.78,526.78,526.78,526.78,526.78,526.78,2019-06-19,1.0,False,NaN
1,9700_-NONE-_SPE7L119P4463_0_-NONE-_0,CONT_AWD_SPE7L119P4463_9700_-NONE-_-NONE-,3796.95,3796.95,3796.95,3796.95,3796.95,3796.95,2019-06-19,1.0,False,NaN
2,9700_-NONE-_SPE7L119P4483_0_-NONE-_0,CONT_AWD_SPE7L119P4483_9700_-NONE-_-NONE-,97.85,97.85,97.85,97.85,97.85,97.85,2019-06-19,1.0,False,NaN
3,9700_-NONE-_SPE7L119P4489_0_-NONE-_0,CONT_AWD_SPE7L119P4489_9700_-NONE-_-NONE-,5602.40,5602.40,5602.40,5602.40,5602.40,5602.40,2019-06-19,1.0,False,NaN
4,9700_-NONE-_SPE7L119V7674_P00001_-NONE-_0,CONT_AWD_SPE7L119V7674_9700_-NONE-_-NONE-,-20406.72,0.00,-20406.72,0.00,-20406.72,0.00,2019-06-19,NaN,True,True


In [21]:
dftest = df_ever_deob[np.logical_and(df_ever_deob['Transaction_Deob'] == False,df_ever_deob['Contract_Deob'] == True)]

# dftest = df.loc[df['Transaction_Deob'] == False]

# dftest2 = dftest.loc[dftest['Contract_Deob']==True]

# np.logical_and(df['transaction'] === false,


dftest.head(10)

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_Deob
153,9700_-NONE-_SPRMM119PPC06_P00001_-NONE-_0,CONT_AWD_SPRMM119PPC06_9700_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.00,0.00,2019-06-19,NaN,False,True
185,9700_-NONE-_W911S219P1402_0_-NONE-_0,CONT_AWD_W911S219P1402_9700_-NONE-_-NONE-,43258.97,0.00,43258.97,0.00,43258.97,0.00,2019-06-19,NaN,False,True
188,9700_-NONE-_W911SA17P3018_P00005_-NONE-_0,CONT_AWD_W911SA17P3018_9700_-NONE-_-NONE-,77472.67,219006.27,77472.67,219006.27,0.00,378631.09,2019-06-19,0.578416,False,True
212,9700_-NONE-_W912P817P0081_P00004_-NONE-_0,CONT_AWD_W912P817P0081_9700_-NONE-_-NONE-,216845.13,295795.61,216845.13,295795.61,0.00,295795.61,2019-06-19,1.000000,False,True
227,9700_9700_0001_20_W91RUS15D0004_0,CONT_AWD_0001_9700_W91RUS15D0004_9700,12019.94,18824050.93,0.00,18843985.57,0.00,32254353.84,2019-06-19,0.583613,False,True
235,9700_9700_0002_1L_W912UM16D0007_0,CONT_AWD_0002_9700_W912UM16D0007_9700,166605.24,3489727.29,166605.24,3489727.29,166605.24,3489727.29,2019-06-19,1.000000,False,True
237,9700_9700_0002_36_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,1696035.00,11214593.50,1696035.00,11214593.50,1696035.00,11214593.50,2019-06-19,1.000000,False,True
238,9700_9700_0002_4_N0010413GA303_0,CONT_AWD_0002_9700_N0010413GA303_9700,0.00,33539.00,0.00,33539.00,0.00,33539.00,2019-06-19,1.000000,False,True
240,9700_9700_0002_89_W52P1J14G0026_1,CONT_AWD_0002_9700_W52P1J14G0026_9700,23311.40,71355403.12,23311.40,71355403.12,23311.40,71355403.12,2019-06-19,1.000000,False,True
244,9700_9700_0003_1H_W9127S13D6005_0,CONT_AWD_0003_9700_W9127S13D6005_9700,9251.00,11565645.16,9251.00,11565645.16,9251.00,11565645.16,2019-06-19,1.000000,False,True


In [34]:
dftest3 = df_ever_deob[df_ever_deob['contract_award_unique_key'] == 'CONT_AWD_0002_9700_W52P1J14D0061_9700'].sort_values(by = 'action_date')

dftest3.head(10)

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_Deob
2900985,9700_9700_0002_33_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,15215.59,8837334.60,15215.59,8837334.60,15215.59,8837334.60,2018-10-24,1.0,False,True
3319239,9700_9700_0002_34_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,-15566.00,8837334.60,-15566.00,8837334.60,-15566.00,8837334.60,2018-11-29,1.0,True,True
5989267,9700_9700_0002_35_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,-4071.00,8833263.60,-4071.00,8833263.60,-4071.00,8833263.60,2019-03-14,1.0,True,True
237,9700_9700_0002_36_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,1696035.00,11214593.50,1696035.00,11214593.50,1696035.00,11214593.50,2019-06-19,1.0,False,True
4331550,9700_9700_0002_37_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,22430.30,11214593.50,22430.30,11214593.50,22430.30,11214593.50,2019-07-02,1.0,False,True
846455,9700_9700_0002_38_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,588493.87,11214593.50,588493.87,11214593.50,588493.87,11214593.50,2019-07-16,1.0,False,True
5075958,9700_9700_0002_39_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,-746.82,11203810.56,-746.82,11203810.56,-746.82,11203810.56,2019-08-06,1.0,False,True
5125735,9700_9700_0002_40_W52P1J14D0061_1,CONT_AWD_0002_9700_W52P1J14D0061_9700,75117.55,11203810.56,75117.55,11203810.56,75117.55,11203810.56,2019-08-07,1.0,False,True


In [31]:
df_test4 = df_ever_deob[df_ever_deob['base_and_all_options_value'] != df_ever_deob['federal_action_obligation']]
print(df_ever_deob.shape)
#6,519,296 rows in df

print(df_test4.shape)
#335,073 where federal action obligatio != base and exercised options value
#352,596 where base & all options != base and exercised options
#300,646 where base & all options value != federal action obligation

df_test4.head()

(6519296, 12)
(300646, 12)


,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_Deob
32,9700_-NONE-_SPE7L519P0806_0_-NONE-_0,CONT_AWD_SPE7L519P0806_9700_-NONE-_-NONE-,1125.54,1125.54,1125.54,1125.54,1254.54,1254.54,2019-06-19,0.897173,False,NaN
152,9700_-NONE-_SPRDL119P0240_0_-NONE-_1,CONT_AWD_SPRDL119P0240_9700_-NONE-_-NONE-,13981.00,13981.00,13981.00,13981.00,38905.00,38905.00,2019-06-19,0.359363,False,NaN
170,9700_-NONE-_W56HZV17PL573_P00003_-NONE-_0,CONT_AWD_W56HZV17PL573_9700_-NONE-_-NONE-,58525.00,170257.00,58525.00,170257.00,0.00,292947.00,2019-06-19,0.581187,False,NaN
172,9700_-NONE-_W81K0019P0266_0_-NONE-_0,CONT_AWD_W81K0019P0266_9700_-NONE-_-NONE-,0.00,0.00,243750.00,243750.00,243750.00,243750.00,2019-06-19,0.000000,False,NaN
173,9700_-NONE-_W81K0216P0189_P00003_-NONE-_0,CONT_AWD_W81K0216P0189_9700_-NONE-_-NONE-,14000.00,56000.00,14000.00,56000.00,0.00,70000.00,2019-06-19,0.800000,False,NaN


In [44]:
df = df.fillna(0)
# Convert action date to pd.datetime
df['action_date'] = pd.to_datetime(df['action_date'])
# set index to each contract and date
df.set_index(['contract_award_unique_key','action_date'], inplace = True)
df.sort_index()

contract_transaction_unique_key  \
contract_award_unique_key              action_date                                                   
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                7022_7022_00001_0_HSFE0408A0017_0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                7022_7022_00001_0_HSFE0408A0035_0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                7022_7022_00001_0_HSFE0408A0066_0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                7022_7022_00001_0_HSFE0408A0067_0   
                                       2012-07-27           7022_7022_00001_P00001_HSFE0408A0067_0   
...                                                                                            ...   
CONT_IDV_TPDFIGBPA100001_2036          2010-10-26   2036_-NONE-_TPDFIGBPA100001_T501_-NONE-_-NONE-   
                                       2010-10-26    2036_-NONE-_TPDFIGBPA100001_500_-NONE-_-NONE-   
CONT_IDV_WRO02005_7012                 2017-05-31          7012_-NONE-_WRO02005_XXX9_-NONE-_-NONE-   
CONT_IDV_WRO9812_7012                  2017-05-31           7012_-NONE-_WRO9812_XXX9_-NONE-_-NONE-   
CONT_IDV_WRO9816_7012                  2017-05-31           7012_-NONE-_WRO9816_XXX9_-NONE-_-NONE-   

                                                    federal_action_obligation  \
contract_award_unique_key              action_date                              
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                     15000.0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                     10000.0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                      5000.0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                      1500.0   
                                       2012-07-27                       -30.0   
...                                                                       ...   
CONT_IDV_TPDFIGBPA100001_2036          2010-10-26                         0.0   
                                       2010-10-26                         0.0   
CONT_IDV_WRO02005_7012                 2017-05-31                         0.0   
CONT_IDV_WRO9812_7012                  2017-05-31                         0.0   
CONT_IDV_WRO9816_7012                  2017-05-31                         0.0   

                                                    total_dollars_obligated  \
contract_award_unique_key              action_date                            
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                       0.0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                       0.0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                       0.0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                       0.0   
                                       2012-07-27                       0.0   
...                                                                     ...   
CONT_IDV_TPDFIGBPA100001_2036          2010-10-26                       0.0   
                                       2010-10-26                       0.0   
CONT_IDV_WRO02005_7012                 2017-05-31                       0.0   
CONT_IDV_WRO9812_7012                  2017-05-31                       0.0   
CONT_IDV_WRO9816_7012                  2017-05-31                       0.0   

                                                    base_and_exercised_options_value  \
contract_award_unique_key              action_date                                     
CONT_AWD_00001_7022_HSFE0408A0017_7022 2008-02-15                            15000.0   
CONT_AWD_00001_7022_HSFE0408A0035_7022 2008-02-20                            10000.0   
CONT_AWD_00001_7022_HSFE0408A0066_7022 2008-05-15                             5000.0   
CONT_AWD_00001_7022_HSFE0408A0067_7022 2008-03-31                             1500.0   
                                       2012-07-27                              -30.0   
...                                       

In [45]:
# We want to only focus on execution rate (for now). I'm not sure which column is the best indicator of execution rate
# we can follow up with Eric. I'm going to stick with Percent Cumulatively Obligated over potential total value of award
# as wequal to the execution rate.
drop_columns = [column for column in df.columns if column != 'Percent Cumulatively Obligated over potential total value of award']
df.drop(drop_columns,axis = 1, inplace=True)

df.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [46]:
df.head(4)

,,execution_rate
contract_award_unique_key,action_date,
CONT_AWD_HSBP20080015201839_7014_GS21F0031U_4730,2008-09-30,0.0
CONT_IDV_HSHQDC06C00065_7001,2008-09-30,0.0
CONT_AWD_HSCG2808P7ABEC9_7008_-NONE-_-NONE-,2008-09-30,0.0
CONT_IDV_HSHQDC08C00190_7001,2008-09-30,0.0


In [47]:
contracts = [contract for contract, date in list(df.index)]
unique_k = set(contracts)
print('This is the number of unique Contracts: ' + str(len(unique_k)))
print('This is the number of transactions: ' + str(len(contracts)))

This is the number of unique Contracts: 485094
This is the number of transactions: 1020089


# Selecting contracts w/ 5 or more transactions from 2008-2018

In [48]:
# I grouped the dataframe by contract and used the .count(method)
# to determine the number of entries. I then used an inequality to get a boolean series of the number of contracts 
# that satisfied the condition of at least 5 transactions.
test = df.groupby(level='contract_award_unique_key')['execution_rate'].count() >= 5
test = test[test==True]
# This is the list of contracts that have more than 5 transactions. 
highlighted_transactions = test.index



### Utilize FY19 Data as Test Data

In [49]:
os.chdir('C:\\Users\\belincoln\\repos\\BudgetPredict')

In [50]:
%cd data

C:\Users\belincoln\repos\BudgetPredict\data


In [51]:
filename = 'FY2019_070_Contracts_Full_20200110_1.csv'

In [52]:
test_df = pd.read_csv(filename, header = 0, usecols = ['contract_transaction_unique_key',
                        'contract_award_unique_key',
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award',
                        'action_date'],
                 dtype = {'contract_transaction_unique_key':'str','contract_transaction_unique_key':'str',
                          'federal_action_obligation': 'float','total_dollars_obligated': 'float', 
                          'base_and_exercised_options_value': 'float', 'current_total_value_of_award': 'float', 
                          'base_and_all_options_value': 'float','potential_total_value_of_award': 'float',
                          'action_date':'str'})

In [53]:
print('This is the # of transactions is FY19 for DHS: ' + str(len(test_df)))

This is the # of transactions is FY19 for DHS: 66533


In [54]:
test_df = test_df.fillna(0)
# Convert action date to pd.datetime
test_df['action_date'] = pd.to_datetime(test_df['action_date'])
# set index to each contract and date
test_df.set_index(['contract_award_unique_key','action_date'], inplace = True)
test_df.sort_index()
fy19_df = test_df.copy()

In [55]:
# We want to only focus on execution rate (for now). I'm not sure which column is the best indicator of execution rate
# we can follow up with Eric. I'm going to stick with Percent Cumulatively Obligated over potential total value of award
# as wequal to the execution rate.
drop_columns = [column for column in test_df.columns if column != 'Percent Cumulatively Obligated over potential total value of award']
test_df.drop(drop_columns,axis = 1, inplace=True)

test_df.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [56]:
contracts = [contract for contract, date in list(test_df.index)]
unique_k_fy19 = set(contracts)
print('This is the number of unique Contracts: ' + str(len(unique_k_fy19)))
print('This is the number of transactions: ' + str(len(contracts)))

This is the number of unique Contracts: 44314
This is the number of transactions: 66533


In [57]:
type(unique_k_fy19)

set

In [58]:
highlighted_transactions = set(highlighted_transactions)

In [59]:
len(unique_k_fy19.intersection(highlighted_transactions))

4022

In [60]:
mask = list(unique_k_fy19.intersection(highlighted_transactions))

In [61]:
fy19_df.index.get_level_values('contract_award_unique_key')

Index(['CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012',
       'CONT_AWD_HSFE2015J0271_7022_HSFE2013D0161_7022',
       'CONT_AWD_70Z02918PX0808600_7008_-NONE-_-NONE-',
       'CONT_AWD_70Z03818PW0000011_7008_-NONE-_-NONE-',
       'CONT_AWD_70Z08018PP45K8500_7008_-NONE-_-NONE-',
       'CONT_IDV_HSCG3816DL00004_7008',
       'CONT_AWD_70Z08418F4YF02400_7008_GS33F0009P_4730',
       'CONT_AWD_HSCG3211J300016_7008_HSCG3210DR00021_7008',
       'CONT_AWD_HSCG3212J300032_7008_HSCG3210DR00021_7008',
       'CONT_IDV_HSTS0514AMED077_7013',
       ...
       'CONT_AWD_HSFE2013MP6330565_7022_-NONE-_-NONE-',
       'CONT_AWD_70Z04019F61427Y00_7008_HSCG4015D60744_7008',
       'CONT_AWD_70Z08519F30211B00_7008_HSCG4015D30002_7008',
       'CONT_AWD_HSTS0415JCT2530_7013_HSHQDC13DE2090_7001',
       'CONT_AWD_70Z03819PD0000080_7008_-NONE-_-NONE-',
       'CONT_AWD_70Z03819PD0000080_7008_-NONE-_-NONE-',
       'CONT_AWD_70Z08019P29577B00_7008_-NONE-_-NONE-',
       'CONT_AWD_HSBP1015F00038_7014_

In [62]:
highlighted_fy19 = fy19_df.loc[(list(highlighted_transactions), slice(None)),:]

In [63]:
highlighted_fy19.head()

,,contract_transaction_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award
contract_award_unique_key,action_date,,,,,,,
CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012,2018-10-19,7012_7012_HSCEMS16F00057_P00005_HSCEMS12A00018_0,-7702.86,6003953.42,-7702.86,6059478.42,-148339.0,6.067181e+06
CONT_IDV_HSCG3816DL00004_7008,2018-10-20,7008_-NONE-_HSCG3816DL00004_P00009_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,3.625838e+06
CONT_IDV_HSCEDM17D00009_7012,2018-10-02,7012_-NONE-_HSCEDM17D00009_P00008_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,4.573615e+08
CONT_IDV_HSCG3815D202051_7008,2018-10-02,7008_-NONE-_HSCG3815D202051_P00005_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,4.074217e+05
CONT_IDV_HSCG4016D60308_7008,2018-10-02,7008_-NONE-_HSCG4016D60308_P00007_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.0,1.818706e+07


In [64]:
highlighted_fy19.index.get_level_values(0).unique()

Index(['CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012',
       'CONT_IDV_HSCG3816DL00004_7008', 'CONT_IDV_HSCEDM17D00009_7012',
       'CONT_IDV_HSCG3815D202051_7008', 'CONT_IDV_HSCG4016D60308_7008',
       'CONT_IDV_HSFLAR16D00001_7015', 'CONT_IDV_HSTS0311DCIO391_7013',
       'CONT_AWD_HSSCCG14F00396_7003_GS03FPM011_4732',
       'CONT_AWD_HSSCCG14F00399_7003_GS03FPM006_4732',
       'CONT_AWD_HSSCCG14Z00017_7003_GS03FPM011_4732',
       ...
       'CONT_AWD_HSBP1012P00838_7014_-NONE-_-NONE-',
       'CONT_AWD_HSFE5015J0268_7022_HSFE5012D0245_7022',
       'CONT_AWD_HSBP1012P00975_7014_-NONE-_-NONE-',
       'CONT_AWD_HSFETX15F0020_7022_GS02F0048M_4730',
       'CONT_AWD_HSCG8513FP45M65_7008_HSCG2313DATB041_7008',
       'CONT_AWD_HSCEDM15P00082_7012_-NONE-_-NONE-',
       'CONT_AWD_70RFP218FREC00009_7001_HSHQEC16D00008_7001',
       'CONT_AWD_HSHQPD06P00002_7001_-NONE-_-NONE-',
       'CONT_AWD_HSHQDC10J00304_7001_HSHQDC06D00024_7001',
       'CONT_AWD_HSSCCG14F00581_7003_HSHQDC1

In [65]:
highlighted_fy19

,,contract_transaction_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award
contract_award_unique_key,action_date,,,,,,,
CONT_AWD_HSCEMS16F00057_7012_HSCEMS12A00018_7012,2018-10-19,7012_7012_HSCEMS16F00057_P00005_HSCEMS12A00018_0,-7702.86,6003953.42,-7702.86,6059478.42,-148339.00,6.067181e+06
CONT_IDV_HSCG3816DL00004_7008,2018-10-20,7008_-NONE-_HSCG3816DL00004_P00009_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.00,3.625838e+06
CONT_IDV_HSCEDM17D00009_7012,2018-10-02,7012_-NONE-_HSCEDM17D00009_P00008_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.00,4.573615e+08
CONT_IDV_HSCG3815D202051_7008,2018-10-02,7008_-NONE-_HSCG3815D202051_P00005_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.00,4.074217e+05
CONT_IDV_HSCG4016D60308_7008,2018-10-02,7008_-NONE-_HSCG4016D60308_P00007_-NONE-_-NONE-,0.00,0.00,0.00,0.00,0.00,1.818706e+07
...,...,...,...,...,...,...,...,...
CONT_AWD_HSHQPD06P00002_7001_-NONE-_-NONE-,2018-10-01,7001_-NONE-_HSHQPD06P00002_P00009_-NONE-_0,0.00,147859.99,0.00,147859.99,0.00,1.478600e+05
CONT_AWD_HSHQDC10J00304_7001_HSHQDC06D00024_7001,2019-09-17,7001_7001_HSHQDC10J00304_P00008_HSHQDC06D00024_0,-448013.23,4977501.44,-448013.23,4977501.44,-448013.23,7.005469e+06
CONT_AWD_HSHQDC15F00108_7001_GS00Q14OADS124_4732,2019-07-09,7001_4732_HSHQDC15F00108_P00025_GS00Q14OADS124_0,505000.00,46688938.83,505000.00,48905927.58,505000.00,4.890593e+07


In [66]:
highlighted_fy19.drop('base_and_exercised_options_value',axis = 1, inplace = True)

In [69]:
df = highlighted_fy19

In [70]:
# Create 3 new features for analysis
df['Percent awarded over potential total awarded'] = df['current_total_value_of_award'] / df['potential_total_value_of_award']
df['Percent Cumulatively Obligated over potential total value of award'] = df['total_dollars_obligated'] / df['potential_total_value_of_award']
df['Percent Cumulatively Obligated over total value already awarded'] = df['total_dollars_obligated'] / df['current_total_value_of_award']

# Create Indicator Variable
df['Indicator'] = df['federal_action_obligation']<-1000

# set index to each transaction key
df.set_index('contract_transaction_unique_key', inplace = True)

C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set 

In [71]:
df.drop('federal_action_obligation', axis =1, inplace = True)
df = df.fillna(0)

In [72]:
# This is the percentage of De-Obligations in the sample (before test/train split)
print('Total number of De-Obligations: ' +str(df['Indicator'].sum()))
print('Percentage of De-Obligations: '+str(df['Indicator'].sum()/len(df)))

Total number of De-Obligations: 1249
Percentage of De-Obligations: 0.14331612162937465


In [73]:
# Create matrix of feature variables and our dependent variable
X, y =  df.iloc[:,1:-1], df.loc[:,'Indicator']

# convert y to np.array of 0s and 1s
y = np.array(y.astype(int))

In [74]:
#train the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [75]:
X_test.shape

(1743, 6)

In [76]:
# Setup the data matrix appropriately, and add ones for the intercept term
m_train, n_train = X_train.shape
m_test, n_test = X_test.shape
# Add intercept term to X
X_train = np.concatenate([np.ones((m_train, 1)), X_train], axis=1)
X_test = np.concatenate([np.ones((m_test, 1)), X_test], axis=1)



In [77]:
log_model = LogisticRegression().fit(X_train, y_train)

C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# Coefficents to multiply features by
log_model.coef_

In [ ]:
theta = log_model.coef_[0]

In [ ]:
print(theta[0])

In [ ]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
        
    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    temp = 1 + np.power(np.e,-z)
    g = 1 / temp
    

    return g

In [ ]:
pred = sigmoid(np.dot(X_test,theta))

In [ ]:
fig = pyplot.figure
pyplot.scatter(np.arange(len(pred)), pred)

In [ ]:
# Now we will test our model using the test_set
pred = sigmoid(np.dot(X_test,theta))

In [ ]:
# Convert probabilty of de-obligation into prediction
pred = pred > .5
# Convert to int
pred = np.array(pred.astype(int))

In [ ]:
test = pred == y_test

In [ ]:
test.sum()/len(test)

In [ ]:
# Really what we are looking for here 
# is how many times a de-obligation occured and we were able to predict it?

In [ ]:
# Also percentage of false positives: how many times did we predict a de-obligation and 
# it did not occur?

In [ ]:
data = {'predicted value' : pred, 'test value': y_test}
testdf = pd.DataFrame(data)

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [ ]:


cm = np.array(confusion_matrix(pred, y_test, labels = [1,0]))
confusion = pd.DataFrame(cm, index=[['Predicted', 'Predicted'], ['Large De-Obligation','Obligation']],
                         columns = [['Test Results', 'Test Results'],['Large De-Obligation','Obligation']])
confusion


In [ ]:
confusion.iloc[0].sum()+confusion.iloc[1].sum()

In [ ]:
correct_preds = testdf[testdf['test value'] == 1]
correct_preds2 = correct_preds[correct_preds['predicted value'] ==1]
print('Size of Test set: ' + str(len(testdf)))
print('Size of Train set: ' + str(len(X_train)))
print('Percentage of correct predictions (Accuracey): ' + str((confusion.iloc[0,0] + confusion.iloc[1,1])/len(testdf)))
print('Percentage of significant de-obligations: ' + str(len(correct_preds)/len(testdf)))
print('Percentage of de-obligations correctly predicted: ' + str(len(correct_preds2)/len(correct_preds)))

In [ ]:
# Rate of False Negatives: A de-obligation occurs, but the model did not predict it. 
false_neg = testdf[testdf['predicted value'] == 0]
false_neg2 = false_neg[false_neg['test value'] == 1]
print('Number of False Negatives : ' + str(len(false_neg2)))
print('Rate of False Negative: ' + str(len(false_neg2)/len(testdf)))

In [ ]:
# Show the rate of False Positives: Times the model predicted a de-obligation, yet there wasn't one

false_pos = testdf[testdf['predicted value'] == 1]
false_pos2 = false_pos[false_pos['test value'] == 0]

print('Number of False Positives: ' + str(len(false_pos2)))
print('Rate of False Positives: ' + str(len(false_pos2)/len(testdf)))